<a href="https://colab.research.google.com/github/q5t8-jw4c-6h9/forge_colab_notebook/blob/main/forge_260118.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Forge環境セットアップ（/content完結・コミット固定0af2869・OpenCVピン）
import os, shutil, glob, subprocess, textwrap

def run(cmd: str, check=True):
    print("+", cmd)
    return subprocess.run(cmd, shell=True, check=check, text=True)

# 0) 作業ディレクトリを強制復旧
os.makedirs("/content", exist_ok=True)
os.chdir("/content")

# 1) 旧フォルダを安全に消去（存在しなくてもOK）
shutil.rmtree("/content/stable-diffusion-webui-forge", ignore_errors=True)

# 2) クローン → 指定コミットに固定（参考コミット：0af2869）
run("git clone https://github.com/q5t8-jw4c-6h9/stable-diffusion-webui-forge /content/stable-diffusion-webui-forge")
os.chdir("/content/stable-diffusion-webui-forge")

# ▼▼▼ 修正：起動時のクローンエラー回避のため先行取得 ▼▼▼
os.makedirs("repositories", exist_ok=True)
run("git clone https://github.com/q5t8-jw4c-6h9/stablediffusion_w-e-w.git repositories/stable-diffusion-stability-ai", check=False)
# ▲▲▲ 修正ここまで ▲▲▲

# 浅いcloneでも確実に固定
run("git fetch origin 0af2869 --depth=1", check=False)
run("git checkout --detach 0af2869", check=False)

# 3) 内蔵拡張の opencv を固定（NumPy 2.x 巻き上げ防止）
paths = glob.glob("extensions-builtin/*/requirements.txt")
if not paths:
    paths = glob.glob("webui/extensions-builtin/*/requirements.txt")  # 互換レイアウト

for p in paths:
    try:
        with open(p, "r", encoding="utf-8") as f:
            s = f.read()
        s2 = s.replace("opencv-python>=4.8.0", "opencv-python==4.11.0.86")
        if s2 != s:
            with open(p, "w", encoding="utf-8") as f:
                f.write(s2)
            print("Pinned:", p)
        else:
            print("Unchanged:", p)
    except FileNotFoundError:
        print("Skip (not found):", p)

print("OK: Cell1 finished.")

+ git clone https://github.com/q5t8-jw4c-6h9/stable-diffusion-webui-forge /content/stable-diffusion-webui-forge
+ git clone https://github.com/q5t8-jw4c-6h9/stablediffusion_w-e-w.git repositories/stable-diffusion-stability-ai
+ git fetch origin 0af2869 --depth=1
+ git checkout --detach 0af2869
Pinned: extensions-builtin/forge_legacy_preprocessors/requirements.txt
Pinned: extensions-builtin/sd_forge_controlnet/requirements.txt
OK: Cell1 finished.


In [2]:
#@title googleドライブマウント
from google.colab import drive
drive.mount('/content/drive')

import os
import shutil

# モデルファイルの保存場所
model_source_dir = "/content/drive/MyDrive/model/"
model_target_dir = "/content/stable-diffusion-webui-forge/models/Stable-diffusion/"

# LoRAファイルの保存場所
lora_source_dir = "/content/drive/MyDrive/LoRA/"
lora_target_dir = "/content/stable-diffusion-webui-forge/models/Lora/"

# ControlNetモデルの保存場所
controlnet_source_dir = "/content/drive/MyDrive/CNmodel/"
controlnet_target_dir = "/content/stable-diffusion-webui-forge/models/ControlNet/"

# 保存先のディレクトリを作成する
os.makedirs(model_target_dir, exist_ok=True)
os.makedirs(lora_target_dir, exist_ok=True)
os.makedirs(controlnet_target_dir, exist_ok=True)

# モデルファイルのコピー
for filename in os.listdir(model_source_dir):
    if filename.endswith(".safetensors"):
        source_file = os.path.join(model_source_dir, filename)
        target_file = os.path.join(model_target_dir, filename)
        if not os.path.exists(target_file):  # ファイルが存在しない場合のみコピー
            shutil.copyfile(source_file, target_file)
            print(f"モデルファイルをコピーしました: {filename}")
        else:
            print(f"モデルファイルは既に存在します: {filename}")

# LoRAファイルのコピー
for filename in os.listdir(lora_source_dir):
    if filename.endswith(".safetensors"):
        source_file = os.path.join(lora_source_dir, filename)
        target_file = os.path.join(lora_target_dir, filename)
        if not os.path.exists(target_file):  # ファイルが存在しない場合のみコピー
            shutil.copyfile(source_file, target_file)
            print(f"LoRAファイルをコピーしました: {filename}")
        else:
            print(f"LoRAファイルは既に存在します: {filename}")

# ControlNetモデルのコピー
for filename in os.listdir(controlnet_source_dir):
    if filename.endswith(".safetensors"):
        source_file = os.path.join(controlnet_source_dir, filename)
        target_file = os.path.join(controlnet_target_dir, filename)
        if not os.path.exists(target_file):  # ファイルが存在しない場合のみコピー
            shutil.copyfile(source_file, target_file)
            print(f"ControlNetモデルをコピーしました: {filename}")
        else:
            print(f"ControlNetモデルは既に存在します: {filename}")

print("すべてのファイルのコピーが完了しました。")

#@markdown ### vae-ft-mse-840000
#@markdown [公式ページ](https://huggingface.co/stabilityai/sd-vae-ft-mse-original)
use_vae_ft_mse_840000 = True #@param {type: "boolean"}
if use_vae_ft_mse_840000:
  !wget -q https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.safetensors --directory-prefix=/content/stable-diffusion-webui-forge/models/VAE/
  !echo "downloaded: vae-ft-mse-840000-ema-pruned.safetensors"

#@markdown ### kl-f8-anime2
#@markdown [公式ページ](https://huggingface.co/hakurei/waifu-diffusion-v1-4)
use_klf8_anime2 = False #@param {type: "boolean"}
if use_klf8_anime2:
  !wget -q https://huggingface.co/hakurei/waifu-diffusion-v1-4/resolve/main/vae/kl-f8-anime2.ckpt --directory-prefix=/content/stable-diffusion-webui-forge/models/VAE/
  !echo "downloaded: kl-f8-anime2.ckpt"

#@markdown ### SDXL - VAE (SDXL向け)
#@markdown [公式ページ](https://huggingface.co/stabilityai/sdxl-vae)
use_sdxl_vae = True #@param {type: "boolean"}
if use_sdxl_vae:
  !wget -q https://huggingface.co/stabilityai/sdxl-vae/resolve/main/sdxl_vae.safetensors --directory-prefix=/content/stable-diffusion-webui-forge/models/VAE/
  !echo "downloaded: sdxl_vae.safetensors"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
モデルファイルをコピーしました: waiIllustriousSDXL_v160.safetensors
モデルファイルをコピーしました: waiSHUFFLENOOB_vPred04.safetensors
ControlNetモデルをコピーしました: CN-anytest4_illustrious2_A.safetensors
すべてのファイルのコピーが完了しました。
downloaded: vae-ft-mse-840000-ema-pruned.safetensors
downloaded: sdxl_vae.safetensors


In [ ]:
#@title forgeの起動
#セル3Forge起動準備（Python 3.10 venv構築）（/content完結）
import os, subprocess

def run(cmd, check=True):
    print("$", cmd)
    return subprocess.run(cmd, shell=True, check=check)

# 既存の venv が 3.10 系で正常ならスキップ
if os.path.exists("/content/py310/bin/python"):
    out = subprocess.run("/content/py310/bin/python -V", shell=True, capture_output=True, text=True)
    if out.returncode == 0 and out.stdout.startswith("Python 3.10."):
        print("[skip] 既存の /content/py310 は Python 3.10 系。再作成をスキップします。")
    else:
        run("rm -rf /content/py310")

if not os.path.exists("/content/py310/bin/python"):
    # 1) APT ミラー対策
    run("sed -i 's|http://archive.ubuntu.com/ubuntu|http://azure.archive.ubuntu.com/ubuntu|g' /etc/apt/sources.list", check=False)
    run("apt-get update -y -o Acquire::Retries=3 -o Acquire::http::Timeout=30", check=False)
    # 2) Python 3.10 導入
    if run("apt-get install -y --no-install-recommends python3.10 python3.10-venv python3.10-distutils", check=False).returncode != 0:
        run("apt-get install -y --no-install-recommends python3.10 python3.10-venv python3-distutils", check=False)
    # 3) venv 作成
    run("python3.10 -m venv /content/py310", check=False)
    # 4) pip ブートストラップ
    if not os.path.exists("/content/py310/bin/pip"):
        run("python3.10 -m ensurepip --upgrade", check=False)
        if not os.path.exists("/content/py310/bin/python"):
            run("rm -rf /content/py310", check=False)
            run("python3.10 -m venv /content/py310", check=False)
        if not os.path.exists("/content/py310/bin/pip"):
            run("curl -sS https://bootstrap.pypa.io/get-pip.py -o /tmp/get-pip.py", check=True)
            run("/content/py310/bin/python /tmp/get-pip.py", check=True)

# 安定版に固定
run("/content/py310/bin/python -m pip install -U 'pip==24.2' 'wheel==0.43.0' 'setuptools==68.2.2'")
run("/content/py310/bin/pip install 'numpy<2'")
run("/content/py310/bin/python --version")

#セル4 PyTorch 2.1.2 + cu121 インストール
#/content 側の venv にインストール
!/content/py310/bin/pip install \
  torch==2.1.2 torchvision==0.16.2 --index-url https://download.pytorch.org/whl/cu121
# xFormers は任意。torch==2.1.2 と相性が良いのは 0.0.23 系。
# !/content/py310/bin/pip install xformers==0.0.23.post1 --index-url https://download.pytorch.org/whl/cu121

#セル5 起動前クリア（venv削除）
# ▼修正: code1.txtに合わせてパスを /content/stable-diffusion-webui-forge に統一
%cd /content/stable-diffusion-webui-forge
# 以前の（Python 3.12系などの）venv は競合するので削除
!rm -rf venv

#セル6 requirements_versions 調整
# 念のためバックアップ（存在時のみ）
!bash -lc 'test -f requirements_versions.txt && cp -n requirements_versions.txt requirements_versions.bak || true'
# jsonmerge 1.8.0 → 1.9.2（存在時のみ）
!bash -lc "test -f requirements_versions.txt && sed -i 's/jsonmerge==1\.8\.0/jsonmerge==1.9\.2/' requirements_versions.txt || true"
# 競合回避（存在時のみ）
!bash -lc "test -f requirements_versions.txt && sed -i '/^protobuf==/d' requirements_versions.txt || true"
!bash -lc "test -f requirements_versions.txt && sed -i '/^Pillow==/d' requirements_versions.txt || true"
# basicsr==1.4.2 はPyPIに無いのでコメントアウト
!bash -lc "test -f requirements_versions.txt && sed -i 's/^basicsr==1\.4\.2/# basicsr==1.4.2 (installed from GitHub)/' requirements_versions.txt || true"

# セル7 SciPy/FilterPy 先行インストール
# ▼修正: 消えていた必須ライブラリ(SciPy/FilterPy)を復元
!/content/py310/bin/pip install --only-binary=:all: "scipy==1.15.3"
!/content/py310/bin/pip install --no-binary=filterpy "filterpy==1.4.5"
# ▼修正: scikit-imageは0.19.3でOK（1.x系ビルド確保のため）
!/content/py310/bin/pip install --only-binary=:all: \
  "scikit-image==0.19.3" \
  "imageio==2.31.1" \
  "tifffile==2023.7.10" \
  "opencv-python==4.11.0.86"

# セル8
# omegaconf のピンをrequirementsから外して明示固定
!bash -lc "test -f requirements_versions.txt && sed -i '/^omegaconf==/d' requirements_versions.txt || true"
# PyYAML不足によるimportエラー修正
!/content/py310/bin/pip install "PyYAML"
!/content/py310/bin/pip install --no-deps "omegaconf==2.3.0"
# antlr は 4.9.3 を先入れ
!/content/py310/bin/pip install --no-binary=:all: "antlr4-python3-runtime==4.9.3"
# BasicSR を依存なしでインストール
!/content/py310/bin/pip install --no-deps "git+https://github.com/xinntao/BasicSR@v1.4.2"
# 念のため numpy<2 を再ピン
!/content/py310/bin/pip install "numpy<2"

# 確認
!/content/py310/bin/python -c "import omegaconf, importlib.metadata, numpy; \
print('omegaconf', omegaconf.__version__); \
print('antlr4', importlib.metadata.version('antlr4-python3-runtime')); \
print('basicsr', importlib.metadata.version('basicsr')); \
print('numpy', numpy.__version__)"

# ▼▼▼ 追加：モデルフォルダが空ならダミーファイルを生成してDLを阻止 ▼▼▼
import os
model_dir = "/content/stable-diffusion-webui-forge/models/Stable-diffusion"
os.makedirs(model_dir, exist_ok=True)

# フォルダ内にモデルファイルがない場合のみ実行
if not any(f.endswith(('.safetensors', '.ckpt')) for f in os.listdir(model_dir)):
    # 0バイトのダミーファイルを作成
    with open(os.path.join(model_dir, "dummy_skip_download.safetensors"), "w") as f:
        f.write("dummy")
    print("生成AI: デフォルトモデルのダウンロードを回避するため、ダミーファイルを設置しました。")
# ▲▲▲ 追加ここまで ▲▲▲

#セル10起動
%cd /content/stable-diffusion-webui-forge

# 念のため、中途半端に作られたフォルダを再度クリア（安全策）
!rm -rf /content/stable-diffusion-webui-forge/repositories/stable-diffusion-stability-ai

# ▼▼▼ 追加: NumPy 2.x をあらゆる手段で阻止する「鉄壁の制約ファイル」を作成 ▼▼▼
!echo "numpy<2" > /content/numpy_constraint.txt
# ▲▲▲ 追加ここまで ▲▲▲

# NumPy 2.x 対策（念のための事前インストール）
!/content/py310/bin/pip install "numpy<2"

# 起動
# ▼修正: PIP_CONSTRAINT 環境変数を追加し、起動中の勝手なアップグレードを禁止
!env -u MPLBACKEND \
  PIP_ONLY_BINARY=":all:" \
  PIP_NO_BINARY="filterpy,antlr4-python3-runtime" \
  PIP_CONSTRAINT="/content/numpy_constraint.txt" \
  LAUNCH_USE_SYSTEM_PYTHON=1 \
  PYTHON=/content/py310/bin/python \
  MPLBACKEND=Agg \
  STABLE_DIFFUSION_REPO=https://github.com/q5t8-jw4c-6h9/stablediffusion_w-e-w.git \
  /content/py310/bin/python launch.py \
    --skip-python-version-check \
    --share \
    --enable-insecure-extension-access \
    --opt-sdp-attention \
    --no-hashing

$ sed -i 's|http://archive.ubuntu.com/ubuntu|http://azure.archive.ubuntu.com/ubuntu|g' /etc/apt/sources.list
$ apt-get update -y -o Acquire::Retries=3 -o Acquire::http::Timeout=30
$ apt-get install -y --no-install-recommends python3.10 python3.10-venv python3.10-distutils
$ python3.10 -m venv /content/py310
$ /content/py310/bin/python -m pip install -U 'pip==24.2' 'wheel==0.43.0' 'setuptools==68.2.2'
$ /content/py310/bin/pip install 'numpy<2'
$ /content/py310/bin/python --version
Looking in indexes: https://download.pytorch.org/whl/cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 GB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 134.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 119.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 121.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 74.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 184.0 

In [ ]:
#@title ダウンロード(google driveマウントしないと落とせない)
# 画像をZIPファイルにまとめる
!zip -r /content/output_images.zip /content/stable-diffusion-webui-forge/output

# ZIPファイルをGoogle Driveにコピーする
import shutil
import os

# 元のZIPファイルのパス
source_path = '/content/output_images.zip'

# Google Driveの保存先パス
drive_path = '/content/drive/MyDrive/output_images.zip'  # 適宜パスを変更してください

# ファイルをコピー
if os.path.exists(source_path):
    shutil.copy(source_path, drive_path)
    print(f"ファイルが正常にGoogle Driveにコピーされました: {drive_path}")
else:
    print(f"エラー: 元のファイル {source_path} が見つかりません。")

# コピーを確認
if os.path.exists(drive_path):
    print("確認: ファイルがGoogle Driveに存在します。")
else:
    print("警告: ファイルがGoogle Driveに見つかりません。")


In [ ]:
#@title google driveマウント　独立　単体
from google.colab import drive
drive.mount('/content/drive')